## やっていること
1. covis_matrix作成
2. candidate_generation

### candidate_generationの方法
- 予め与えられているaidを候補にする
- 与えられているaidの最後からcovis_matrixを使って候補を作る

In [2]:
# covismatrixを作る
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
personal_access_token = user_secrets.get_secret("personal_access_token")

!rm -rf /kaggle/working/kaggle_otto
!git clone https://$personal_access_token@github.com/coffeemountain/kaggle_otto.git

import sys
sys.path.append('/kaggle/working/kaggle_otto/src')

# 好きなcomponentをimportする
from covis_matrix_generator import *

Cloning into 'kaggle_otto'...
remote: Enumerating objects: 147, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 147 (delta 31), reused 23 (delta 14), pack-reused 92
Receiving objects: 100% (147/147), 98.29 KiB | 571.00 KiB/s, done.
Resolving deltas: 100% (64/64), done.


In [3]:
import glob

files = glob.glob('../input/otto-chunk-data-inparquet-format/*_parquet/*')

USE_GPU = True
weight_func_mixin = WeightFuncMixin()
covis_matrix_generator = CovisMatrixGenerator(weight_func_mixin, use_gpu=USE_GPU)

click_config = Config(
    target_types=[0, 1, 2], 
    weight_func='type_weight_1_6_3',
    min_event_threshold=30,
    max_sec_threshold=24 * 60 * 60,
    save_topk=15,
    output_dir='click'
)
covis_matrix_generator.generate(click_config, files)

### DISK PART 1
Processing files 0 thru 24 in groups of 5...


/opt/conda/lib/python3.7/site-packages/cudf/core/frame.py:2651: UserWarning: When using a sequence of booleans for `ascending`, `na_position` flag is not yet supported and defaults to treating nulls as greater than all numbers
  "When using a sequence of booleans for `ascending`, "


0 , 5 , 10 , 15 , 20 , 
Processing files 25 thru 49 in groups of 5...
25 , 30 , 35 , 40 , 45 , 
Processing files 50 thru 74 in groups of 5...
50 , 55 , 60 , 65 , 70 , 
Processing files 75 thru 99 in groups of 5...
75 , 80 , 85 , 90 , 95 , 
Processing files 100 thru 124 in groups of 5...
100 , 105 , 110 , 115 , 120 , 
Processing files 125 thru 145 in groups of 5...
125 , 130 , 135 , 140 , 145 , 
### DISK PART 2
Processing files 0 thru 24 in groups of 5...
0 , 5 , 10 , 15 , 20 , 
Processing files 25 thru 49 in groups of 5...
25 , 30 , 35 , 40 , 45 , 
Processing files 50 thru 74 in groups of 5...
50 , 55 , 60 , 65 , 70 , 
Processing files 75 thru 99 in groups of 5...
75 , 80 , 85 , 90 , 95 , 
Processing files 100 thru 124 in groups of 5...
100 , 105 , 110 , 115 , 120 , 
Processing files 125 thru 145 in groups of 5...
125 , 130 , 135 , 140 , 145 , 
### DISK PART 3
Processing files 0 thru 24 in groups of 5...
0 , 5 , 10 , 15 , 20 , 
Processing files 25 thru 49 in groups of 5...
25 , 30 , 35

In [4]:
del covis_matrix_generator, files

In [5]:
data_dir = '/kaggle/input/otto-train-and-test-data-for-local-validation'
df_val = pd.read_parquet(f'{data_dir}/test.parquet')

In [6]:
def pqt_to_dict(df):
    return df.groupby('aid_x').aid_y.apply(list).to_dict()

# LOAD THREE CO-VISITATION MATRICES
path_to_dir = '/kaggle/working/click'
top_20_buys = pqt_to_dict( pd.read_parquet(f'{path_to_dir}/part_0.pqt') )
for k in range(1,4): 
    top_20_buys.update( pqt_to_dict( pd.read_parquet(f'{path_to_dir}/part_{k}.pqt') ) )


In [7]:
# step1
## 与えられたitemと1つのcovis_matrix
def generate_candidate(x, k=50):
    # 与えられたitem列をそのまま候補に
    candidates = x.aid.values.tolist()
    
    # 最後のitemからcovis_matrixを使って、候補を増やす
    last_item = candidates[-1]
    candidates.extend(top_20_buys.get(last_item, []))
    
    # 最大50まで
    return candidates[:k]

In [8]:
from tqdm import tqdm
tqdm.pandas()

candidates = df_val.groupby('session').progress_apply(lambda x: generate_candidate(x)).explode()
# candidates
candidates

100%|██████████| 1801251/1801251 [01:25<00:00, 21000.41it/s]


session
11098528      11830
11098528    1732105
11098528     588923
11098528     876129
11098528     884502
             ...   
12899778      32070
12899778     379804
12899778    1097813
12899778    1102546
12899778    1320960
Length: 34057457, dtype: object

In [9]:
candidates.name = 'item'
candidates = candidates.to_frame().reset_index()
candidates

,session,item
0,11098528,11830
1,11098528,1732105
2,11098528,588923
3,11098528,876129
4,11098528,884502
...,...,...
34057452,12899778,32070
34057453,12899778,379804
34057454,12899778,1097813
34057455,12899778,1102546
